In [2]:
from pathlib import Path
from hanziconv import HanziConv

In [28]:
# Preprocess kamyung folder
p = Path("./kamyung")
files = p.iterdir()
content : str = ''
for fn in files:
    print(fn)
    with open(fn, 'r', encoding='gbk', errors='ignore') as fp:
        content += HanziConv.toTraditional("\n".join(fp.readlines()[5:]))
with open("./train.txt", "w", encoding='utf-8') as fp:
    fp.write(content)


kamyung/侠客行.txt
kamyung/射雕英雄传.txt
kamyung/鹿鼎记.txt
kamyung/雪山飞狐.txt
kamyung/连城诀.txt
kamyung/飞狐外传.txt
kamyung/神雕侠侣.txt
kamyung/天龙八部.txt
kamyung/碧血剑.txt
kamyung/白马啸西风.txt
kamyung/笑傲江湖.txt
kamyung/倚天屠龙记.txt
kamyung/书剑恩仇录.txt
